# Linear Algebra

## Elimination

Elimination of linear algebra is the basic process of eliminating all the variables from a equation that you have 1 variable left. For example, if you have an equation with X, Y, Z variables, you can use the other equations to knockout X, Y variables such that you have Z left. 

In most text, they will teach to use a certain position as a pivot, but it really doesn't matter. Just do whatever it takes to eliminate the variables, you can subtract from any position as long as it serves the purpose in eliminating the variables.

Moreover, you can observe that once you have eliminated one equation to only have 1 variable, you can then use that equation to eliminate the other equations. For example, if you have an equation with 0X + 0Y + 2Z, this becomes a powerful tool to eliminate Z from the other 2 equations because X and Y is zero and has no effect. In the end, you keep doing eliminations until you have only 1 variable per equation.

### Failures in elimination

Intuitively, elimination will fail when you eliminate more than 1 variable when you subtract. This will cause it to fail because you will end up with a eqution with X isolated, Y isolated, but none with Z isolated, because you accidentally eliminated Z together with another variable. This problem occurs because two equations are not independent and they are just derivations of another. In this case, you really only have information for two equations, so it is natural that you don't have enough information to solve for all three variables.

### In Row Form

#### Row Operations with matrix

Row operations with matrix happen on a simple principle. The params are on the left hand side and the matrix you want to operate on is on the right handside.

For example:

[1 0 0] [[1 2 3] [4 5 6] [7 8 9]] 
basically means, take 1 x row 1, take 0 x row 2, and take 0 x row 3.

Note that when you multiply a row by a matrix, you get a row back.
Now, going back to eliminations, we can use a matrix as the operator on the matrix instead.
Every row in the matrix operator decides on the corresponding row of the result. Every column in the matrix operator decides on how we want to use the surrounding rows to obtain the result.

Take the identity matrix:
[[1 0 0] [0 1 0] [0 0 1]] x [[1 2 3] [4 5 6] [7 8 9]] = [[1 2 3] [4 5 6] [7 8 9]]

To narrate through what is happening,
The first row of the operator decides on what the first row of the answer is. In this case, we said we take the 1 x first row of the matrix, and add it with 0 of the second row and 0 of the thrid row, meaning that we get back the first row for the answer.
In the second row of the operator, we said that we take 0 x first row, 1 x second row, 0 x third row, meaning get we get back second row of the matrix for the second row of the answer.
In the third row of the operator, we said that we take 0 x first, row, 0 x second row, 1 x third row, meaning get we get back third row of the matrix for the third row of the answer.

#### In code

In [ ]:
def mat_invert(mat):
    inverted_mat = []
    for col_index, _ in enumerate(mat[0]):
        inv_row = []
        for row in mat:
            inv_row.append(row[col_index])
        inverted_mat.append(inv_row)

    return inverted_mat

def mat_add_row(mat):
    summed_mat = []
    for row in mat:
        row_sum = 0
        for column in row:
            row_sum += column
        summed_mat.append(row_sum)

    return summed_mat

def mat_add_col(mat):
    inverted_mat = mat_invert(mat)
    return mat_add_row(inverted_mat)


def mat_mul(mat1, mat2):
    final_mat = []
    for row_index, row in enumerate(mat1):

        new_rows = [] # the new row at row_index of the answer before collapsing
        for col_index, col_val in enumerate(row):
            # Identified the col_val and row location: col_index
            # Note that col_index of operator is the row_index of the target mat
            new_row = []
            for row_val in mat2[col_index]:
                new_row.append(col_val * row_val)
            
            new_rows.append(new_row)
        
        # Add the columns together
        final_row = mat_add_col(new_rows)
        final_mat.append(final_row)
    return final_mat

def find_zeroes(mat):
    zeroes = []
    for row_index, row in enumerate(mat):
        for column_index, column in enumerate(row):
            if column == 0:
                zeroes.append((row_index, column_index))
    return zeroes

def get_next_target_from_curr_target(index, shape):
    row_index, col_index = index
    max_row, max_col = shape

    if row_index + 1 > max_row:
        return col_index + 1 + 1 , col_index + 1

    return row_index + 1, col_index

def find_next_target(zeroes, shape):
    sorted_zeroes = sorted(zeroes, key=lambda zero_index: zero_index[1])

    nxt_target = None
    curr_zero = sorted_zeroes[0]
    while True:
        nxt_target = get_next_target_from_curr_target(curr_zero, shape)
        if nxt_target in sorted_zeroes:
            sorted_zeroes.pop(0)
            curr_zero = sorted_zeroes[0]
        else:
            break
    return nxt_target

def gen_identity_mat(mat):
    max_row, max_col = len(mat) - 1, len(mat[0]) - 1
    i_mat = []
    for i_col in range(max_col + 1):
        row = []
        for i_row in range(max_row + 1):
            if i_row == i_col:
                row.append(1)
            else:
                row.append(0)
        i_mat.append(row)
    return i_mat

def subset_mat(mat):
    subset_mat = []
    for row_i, row in enumerate(mat):
        subset_row = []
        for col_i, col in enumerate(row):
            if row_i == 0 or col_i == 0:
                continue
            subset_row.append(col)
        if subset_row:
            subset_mat.append(subset_row)

    return subset_mat

def superset_mat(mat, super_row, super_col):
    superset = []
    for i_col, col_val in enumerate(super_col):
        if i_col == 0:
            superset.append(super_row)
            continue
        row = []
        for i_row, row_val in enumerate(super_row):
            if i_row == 0:
                row.append(col_val)
            else:
                row.append(mat[i_col - 1][i_row - 1])
        superset.append(row)

    return superset
    


def eliminate_mat(mat, b):
    # pre-process to make sure the first pivot point is not zero
    if mat[0][0] == 0 and mat[1][0] != 0:
        temp = mat[1]
        mat[1] = mat[0]
        mat[0] = temp

    pivot = mat[0][0]
    curr_mat = mat
    for row_i, row_val in enumerate(mat_invert(mat)[0]):
        if row_i == 0: continue

        param = row_val / pivot
        
        # generate the first operator
        operator = gen_identity_mat(mat)
        operator[row_i][0] = -param

        curr_mat = mat_mul(operator, mat_invert([*mat_invert(curr_mat), b]))
        curr_mat = mat_invert(curr_mat)
        b = curr_mat.pop()
        curr_mat = mat_invert(curr_mat)

    max_row, max_col = len(mat) - 1, len(mat[0]) - 1

    if max_col > 1 and max_row > 1:
        first_row = curr_mat[0]
        first_column = mat_invert(curr_mat)[0]

        mat_subset = subset_mat(curr_mat)
        eliminated_subset, b_subset = eliminate_mat(mat_subset, b[1:])
        return superset_mat(eliminated_subset, first_row, first_column), [b[0], *b_subset]
    else:
        return curr_mat, b
                
def back_substitution(mat, b):
    b_reverse = b[::-1]
    answers = [None] * len(mat)
    for eq_i, eq in enumerate(mat[::-1]):
        lhs_sum = 0
        var_index = None
        for ans_i, ans in enumerate(answers):
            if ans is None and eq[ans_i] != 0:
                var_index = ans_i
            elif ans is None and eq[ans_i] == 0:
                lhs_sum += 0
            else:
                lhs_sum += eq[ans_i] * ans
        new_ans = (b_reverse[eq_i] - lhs_sum) / eq[var_index]
        answers[var_index] = new_ans
    return answers


def solve_mat(mat, b):
    eliminated_mat, eliminated_b = eliminate_mat(mat, b)
    result = back_substitution(eliminated_mat, eliminated_b)
    return result
